## Salary Prediction

### Table of Contents

1. [Problem Statement](#Problem-Statement)
2. [Libraries](#Libraries)
3. [Data Loading](#Data-Loading)
4. [Exploratory Data Analysis](#Exploratory-Data-Analysis)
   - [Dataset Summary](#Dataset-Summary)
   - [Data Quality Assessment](#Data-Quality-Assessment)
        - [Missing Values](#Missing-Values)
        - [Duplicates](#Duplicates)
   - [Dataset Distribution](#Dataset-Distribution)
5. [Data Preprocessing](#Data-Preprocessing)
6. [Model Building, Training and Hyperparameter Tuning](#Model-Building-and-Training)
     1. [Model 1: Random Forest](#Model-1:-Random-Forest)
     2. [Model 2: Linear Regression](#Model-2:-Linear-Regression)
7. [Model Evaluation](#Model-Evaluation)
8. [Results and Insights](#Results-and-insights)
9.  [Future Work](#Future-Work)
10. [Conclusion](#Conclusion)


#### Problem Statement

#### Libraries

In [1]:
import pandas as pd
import plotly.express as px
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from tabulate import tabulate
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from helper import header_manipulation, check_unique_values

#### Data Loading

In [2]:
# load the dataset
salary_df = pd.read_csv("data/salary_data.csv")

# display the columns
salary_df.columns

Index(['Unnamed: 0', 'Job Title', 'Location', 'Size', 'Type of ownership',
       'Industry', 'Sector', 'hourly', 'employer_provided', 'min_salary',
       'max_salary', 'avg_salary', 'job_state', 'python_yn', 'R_yn', 'spark',
       'aws', 'excel', 'job_simp', 'seniority'],
      dtype='object')

#### Exploratory Data Analysis

##### Dataset Summary

In [3]:
# display the dataset's shape
print(f"Dataset's shape: {salary_df.shape}\n")

# display the summary of the dataset
print("Dataset Description:")
print(salary_df.describe())

# display the information - column data types - of the dataset
print("\nDataset info:")
print(salary_df.info())

# display the number of categorical and numerical columns
num_categorical = salary_df.select_dtypes(include='object').shape[1]
num_numerical = salary_df.select_dtypes(include=['int64', 'float64']).shape[1]
print(f"\nNumber of categorical columns: {num_categorical}")
print(f"Number of numerical columns: {num_numerical}")


Dataset's shape: (742, 20)

Dataset Description:
       Unnamed: 0      hourly  employer_provided  min_salary  max_salary  \
count  742.000000  742.000000         742.000000  742.000000  742.000000   
mean   370.500000    0.032345           0.022911   74.719677  128.149596   
std    214.341239    0.177034           0.149721   30.980593   45.220324   
min      0.000000    0.000000           0.000000   15.000000   16.000000   
25%    185.250000    0.000000           0.000000   52.000000   96.000000   
50%    370.500000    0.000000           0.000000   69.500000  124.000000   
75%    555.750000    0.000000           0.000000   91.000000  155.000000   
max    741.000000    1.000000           1.000000  202.000000  306.000000   

       avg_salary   python_yn        R_yn       spark         aws       excel  
count  742.000000  742.000000  742.000000  742.000000  742.000000  742.000000  
mean   100.626011    0.528302    0.002695    0.225067    0.237197    0.522911  
std     38.855948    0.499

**Observations from the results:**
1. The dataset contains 742 rows and 20 columns, with the first column serving as the unique identifier for each row.
2. There are 9 categorical columns and 11 numerical columns.
3. The target variable is the `avg_salary` column, which is a numerical column.
4. Some columns are indicated as `numeric`, but they are `boolean` columns. These columns are `python`, `r`, `spark`, `aws`, `excel`, `job_simp`, `seniority`, `desc_len`, `num_comp`. They indicate whether the skill is required or not.

##### Dataset Quality Assessment

In [4]:
# check for missing values
missing_values = salary_df.isna().sum()
print(f"Dataset Missing values count:")
print(missing_values)

# check for duplicates
duplicates = salary_df[salary_df.duplicated()].count()
print(f"Dataset Duplicates count:")
print(duplicates)

Dataset Missing values count:
Unnamed: 0           0
Job Title            0
Location             0
Size                 0
Type of ownership    0
Industry             0
Sector               0
hourly               0
employer_provided    0
min_salary           0
max_salary           0
avg_salary           0
job_state            0
python_yn            0
R_yn                 0
spark                0
aws                  0
excel                0
job_simp             0
seniority            0
dtype: int64
Dataset Duplicates count:
Unnamed: 0           0
Job Title            0
Location             0
Size                 0
Type of ownership    0
Industry             0
Sector               0
hourly               0
employer_provided    0
min_salary           0
max_salary           0
avg_salary           0
job_state            0
python_yn            0
R_yn                 0
spark                0
aws                  0
excel                0
job_simp             0
seniority            0
dtype: int6

##### Dataset Distribution

In [5]:
# function to display the unique values in all columns
def check_unique_values(dataframe: pd.DataFrame):
    """display the unique values in categorical columns and columns with less than 10 unique values

        :Input:
            dataframe: a python pandas dataframe

        :Output:
            unique values in the dataframe

        >>> check_unique_values(salary_prediction_dataframe)
        Unique values in 'spark' column: False, True
        """
    
    # Iterate over each column in the DataFrame
    column_list = []

    index = 0
    for column in dataframe.columns:
        if dataframe[column].dtype == 'object' or len(dataframe[column].unique()) < 10:
            unique_values = dataframe[column].unique()
            unique_values_str = [str(value) for value in unique_values]
            print(
                f"Unique values in '{column}' column: {', '.join(unique_values_str)}")
            column_list.append(index)
            index += 1

    # return column_list


check_unique_values(salary_df)

# This helps identify the categorical and boolean columns

Unique values in 'Job Title' column: Data Scientist, Healthcare Data Scientist, Research Scientist, Staff Data Scientist - Technology, Data Analyst, Data Engineer I, Scientist I/II, Biology, Customer Data Scientist, Data Scientist - Health Data Analytics, Senior Data Scientist / Machine Learning, Data Scientist - Quantitative, Digital Health Data Scientist, Associate Data Analyst, Clinical Data Scientist, Data Scientist / Machine Learning Expert, Web Data Analyst, Senior Data Scientist, Data Engineer, Data Scientist - Algorithms & Inference, Scientist, Lead Data Scientist, Spectral Scientist/Engineer, College Hire - Data Scientist - Open to December 2019 Graduates, Data Scientist, Office of Data Science, Data Science Analyst, Senior Risk Data Scientist, Data Scientist in Artificial Intelligence Early Career, Data Scientist - Research, R&D Data Analysis Scientist, Analytics Consultant, Director, Data Science, Data Scientist SR, R&D Sr Data Scientist, Customer Data Scientist/Sales Engine

In [6]:
# set the boolean columns
boolean_columns = ['hourly', 'employer_provided', 'python_yn', 'R_yn', 'spark', 'aws', 'excel']

In [7]:
# display correlation heatmaps for the numerical columns
numerical_columns = salary_df.select_dtypes(include=['int64', 'float64']).columns

# exclude the boolean columns
numerical_columns = numerical_columns.difference(boolean_columns)

correlation_matrix = salary_df[numerical_columns].corr()

fig = px.imshow(correlation_matrix, text_auto=True, width=800, height=600)
fig.show()

# using matplotlib
# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix)
# plt.show()


In [8]:
# check the value counts for seniority
salary_df['seniority'].value_counts()

seniority
na        520
senior    220
jr          2
Name: count, dtype: int64

In [9]:
# check the value counts for the job_simp
salary_df['job_simp'].value_counts()

job_simp
data scientist    279
na                184
data engineer     119
analyst           102
manager            22
mle                22
director           14
Name: count, dtype: int64

In [10]:
# display the count of values where their seniority is 'na' and the job simp is 'na'
salary_df[(salary_df['seniority'] == 'na') & (salary_df['job_simp'] == 'na')].count()

Unnamed: 0           127
Job Title            127
Location             127
Size                 127
Type of ownership    127
Industry             127
Sector               127
hourly               127
employer_provided    127
min_salary           127
max_salary           127
avg_salary           127
job_state            127
python_yn            127
R_yn                 127
spark                127
aws                  127
excel                127
job_simp             127
seniority            127
dtype: int64

In [11]:
# check the average salary, grouping by the job title

# first get the unique job titles
# len(salary_df["Job Title"].unique())
# len(salary_df['job_simp'].unique())
salary_df["avg_salary"].groupby(salary_df['Job Title']).value_counts()

Job Title                                           avg_salary
Ag Data Scientist                                   80.5          1
Analytics - Business Assurance Data Analyst         43.0          2
Analytics Consultant                                66.5          1
Analytics Manager                                   87.5          2
Analytics Manager - Data Mart                       64.0          4
                                                                 ..
System and Data Analyst                             59.0          1
Systems Engineer II - Data Analyst                  62.5          2
Technology-Minded, Data Professional Opportunities  70.5          2
VP, Data Science                                    124.5         2
Web Data Analyst                                    106.0         1
Name: count, Length: 432, dtype: int64

In [12]:
# display the Type of Ownership
values = salary_df['Type of ownership'].value_counts(
    ascending=False).values

keys = salary_df['Type of ownership'].value_counts(
    ascending=False).keys()

bar_chart = px.bar(x=keys, y=values, color=values, text=values)

bar_chart.update_layout(
    yaxis_title="Type of Ownership",
    xaxis_title="Count",
    title="Type of Ownership Distribution",
)

bar_chart.show()

In [13]:
# display the Sector
values = salary_df['Sector'].value_counts(
    ascending=False).values

keys = salary_df['Sector'].value_counts(
    ascending=False).keys()

bar_chart = px.bar(x=keys, y=values, color=values, text=values)

bar_chart.update_layout(
    yaxis_title="Sector",
    xaxis_title="Count",
    title="Sector Distribution",
)

bar_chart.show()

In [14]:
# display the Industry distribution
values = salary_df['Industry'].value_counts(
    ascending=False).values

keys = salary_df['Industry'].value_counts(
    ascending=False).keys()

bar_chart = px.bar(x=keys, y=values, color=values, text=values)

bar_chart.update_layout(
    yaxis_title="Industry",
    xaxis_title="Count",
    title="Industry Distribution",
)

bar_chart.show()

In [15]:
# display the Size
values = salary_df['Size'].value_counts(
    ascending=False).values

keys = salary_df['Size'].value_counts(
    ascending=False).keys()

bar_chart = px.bar(x=keys, y=values, color=values, text=values)

bar_chart.update_layout(
    yaxis_title="Size",
    xaxis_title="Count",
    title="Size Distribution",
)

bar_chart.show()

In [16]:
# Calculate average salary for each seniority group
average_salaries = salary_df.groupby('seniority')['avg_salary'].mean().reset_index()

# Create a bar chart
bar_chart = px.bar(
    average_salaries,
    x='seniority', 
    y='avg_salary',
    text='avg_salary',
    color='avg_salary',
    title="Average Salary by Seniority"
)

# Customize layout
bar_chart.update_layout(
    xaxis_title="Seniority",
    yaxis_title="Average Salary",
    legend_title="Average Salary"
)

bar_chart.show()


#### Data Preprocessing

##### Column Headers Manipulation

In [17]:
# put all columns into lowercase, replace spaces with underscores
salary_df.columns = header_manipulation(salary_df)

# drop the unnamed column
salary_df = salary_df.drop(columns=['unnamed:_0'])

In [18]:
# ensure the boolean columns are present in the DataFrame
boolean_columns = pd.Index(boolean_columns).str.lower()
boolean_columns = boolean_columns.intersection(salary_df.columns)

# change the data types for the boolean columns
salary_df[boolean_columns] = salary_df[boolean_columns].astype(bool)

salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   job_title          742 non-null    object 
 1   location           742 non-null    object 
 2   size               742 non-null    object 
 3   type_of_ownership  742 non-null    object 
 4   industry           742 non-null    object 
 5   sector             742 non-null    object 
 6   hourly             742 non-null    bool   
 7   employer_provided  742 non-null    bool   
 8   min_salary         742 non-null    int64  
 9   max_salary         742 non-null    int64  
 10  avg_salary         742 non-null    float64
 11  job_state          742 non-null    object 
 12  python_yn          742 non-null    bool   
 13  r_yn               742 non-null    bool   
 14  spark              742 non-null    bool   
 15  aws                742 non-null    bool   
 16  excel              742 non

##### Column Transformation

In [19]:
# set the categorical columns
categorical_columns = salary_df.select_dtypes(include='object').columns

# set the updated numerical columns
numerical_columns = salary_df.select_dtypes(include=['int64', 'float64']).columns

# set the updated boolean columns
boolean_columns = salary_df.select_dtypes(include='bool').columns

print(f"Categorical columns length: {len(categorical_columns)} \nNumerical columns length: {len(numerical_columns)} \nBoolean columns length: {len(boolean_columns)}")
print(f"Categorical columns: {categorical_columns} \nNumerical columns: {numerical_columns} \nBoolean columns: {boolean_columns}")

Categorical columns length: 9 
Numerical columns length: 3 
Boolean columns length: 7
Categorical columns: Index(['job_title', 'location', 'size', 'type_of_ownership', 'industry',
       'sector', 'job_state', 'job_simp', 'seniority'],
      dtype='object') 
Numerical columns: Index(['min_salary', 'max_salary', 'avg_salary'], dtype='object') 
Boolean columns: Index(['hourly', 'employer_provided', 'python_yn', 'r_yn', 'spark', 'aws',
       'excel'],
      dtype='object')


In [20]:
# Preprocess features using Column Transformer and Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        # ('num', StandardScaler(), numerical_columns),
        ('bool', 'passthrough', boolean_columns)
    ]
)

# Pipeline with Linear Regression
lr_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('lr_model', LinearRegression())
])

# Pipeline with Random Forest Regressor
rf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('rf_model', RandomForestRegressor(n_estimators=100, random_state=42))
])

#### Model Building, Training and Hyperparameter Tuning

In [21]:
# set the features and the target
X = salary_df.drop(columns=['avg_salary'])
y = salary_df['avg_salary']

# set the stratified Kflod
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [22]:
# Cross-validation for Linear Regression
lr_scores = cross_validate(
    estimator=lr_pipeline,
    X=X,
    y=y,
    cv=kfold,
    scoring=['neg_mean_squared_error', 'r2'],
)

# get the MSE
lr_mse = f"{-lr_scores['test_neg_mean_squared_error'].mean():.3f}"

# get the R²
lr_r2 = f"{lr_scores['test_r2'].mean():.3f}"


# Cross-validation for Random Forest Regressor
rf_scores = cross_validate(
    estimator=rf_pipeline,
    X=X,
    y=y,
    cv=kfold,
    scoring=['neg_mean_squared_error', 'r2']
)

# get the MSE
rf_mse = f"{-rf_scores['test_neg_mean_squared_error'].mean():.3f}"

# get the R²
rf_r2 = f"{rf_scores['test_r2'].mean():.3f}"


# display the results in a table

results = [
    ["Linear Regression", lr_mse, lr_r2],
    ["Random Forest", rf_mse, rf_r2]
]

# Print the table
print(tabulate(results, headers=["Model Name", "MSE", "R²"], tablefmt="pretty"))

+-------------------+---------+-------+
|    Model Name     |   MSE   |  R²   |
+-------------------+---------+-------+
| Linear Regression | 534.887 | 0.624 |
|   Random Forest   | 330.644 | 0.766 |
+-------------------+---------+-------+


In [23]:
# Using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Linear Regression
lr_pipeline.fit(X_train, y_train)

# predict using the test set
y_pred_lr = lr_pipeline.predict(X_test)

# Evaluate
lr_mse = mean_squared_error(y_test, y_pred_lr)
lr_r2 = r2_score(y_test, y_pred_lr)

# Random Forest
rf_pipeline.fit(X_train, y_train)

# predict using the test set
y_pred_rf = rf_pipeline.predict(X_test)

# Evaluate
rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_r2 = r2_score(y_test, y_pred_rf)

# display the results in a table
results = [
    ["Linear Regression", lr_mse, lr_r2],
    ["Random Forest", rf_mse, rf_r2]
]

# Print the table
print(tabulate(results, headers=["Model Name", "MSE", "R²"], tablefmt="pretty"))


+-------------------+-------------------+---------------------+
|    Model Name     |        MSE        |         R²          |
+-------------------+-------------------+---------------------+
| Linear Regression | 919.8219853236259 | 0.40812172055766505 |
|   Random Forest   | 436.019149103139  | 0.7194345559328215  |
+-------------------+-------------------+---------------------+


**Observation:**
- From both `cross_validate` and `train_test_split` results, the Random Forest model outperforms the Linear Regression model.
- The Random Forest model has a higher R2 score and lower RMSE value compared to the Linear Regression model.
- This indicates that the Random Forest model is a better model for predicting the salary of a job.

In [24]:
# Hyperparameter tuning using GridSearchCV

# Define hyperparameter grids
lr_param_grid = {
    'lr_model__fit_intercept': [True, False]
}

rf_param_grid = {
    'rf_model__n_estimators': [50, 100, 200],
    'rf_model__max_depth': [5, 10, 20, None],
    'rf_model__min_samples_split': [2, 5, 10],
    'rf_model__min_samples_leaf': [1, 2, 4]
}

# GridSearch for Linear Regression
lr_search = GridSearchCV(
    estimator=lr_pipeline,
    param_grid=lr_param_grid,
    cv=kfold,
    scoring='neg_mean_squared_error',
    # verbose=2,
    n_jobs=-1
)

# GridSearch for Random Forest Regressor
rf_search = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=rf_param_grid,
    cv=kfold,
    scoring='neg_mean_squared_error',
    # verbose=2,
    n_jobs=-1
)

# Fit the GridSearchCV objects
print("Tuning Linear Regression...")
lr_search.fit(X, y)

print("Tuning Random Forest Regressor...")
rf_search.fit(X, y)

# Best parameters and scores
print("Best parameters for Linear Regression:", lr_search.best_params_)
print("Best MSE for Linear Regression:", -lr_search.best_score_)

print("Best parameters for Random Forest Regressor:", rf_search.best_params_)
print("Best MSE for Random Forest Regressor:", -rf_search.best_score_)


Tuning Linear Regression...
Tuning Random Forest Regressor...
Best parameters for Linear Regression: {'lr_model__fit_intercept': False}
Best MSE for Linear Regression: 534.1975503180103
Best parameters for Random Forest Regressor: {'rf_model__max_depth': None, 'rf_model__min_samples_leaf': 1, 'rf_model__min_samples_split': 2, 'rf_model__n_estimators': 200}
Best MSE for Random Forest Regressor: 325.41242810006895


In [25]:
# Retrieve best parameters for Linear Regression
best_lr_params = lr_search.best_params_

# Create a new pipeline with the best parameters for Linear Regression
lr_pipeline_tuned = Pipeline([
    ('preprocessing', preprocessor),
    ('lr_model', LinearRegression(
        fit_intercept=best_lr_params['lr_model__fit_intercept']
    ))
])

# Retrieve best parameters for Random Forest Regressor
best_rf_params = rf_search.best_params_

# Create a new pipeline with the best parameters for Random Forest Regressor
rf_pipeline_tuned = Pipeline([
    ('preprocessing', preprocessor),
    ('rf_model', RandomForestRegressor(
        n_estimators=best_rf_params['rf_model__n_estimators'],
        max_depth=best_rf_params['rf_model__max_depth'],
        min_samples_split=best_rf_params['rf_model__min_samples_split'],
        min_samples_leaf=best_rf_params['rf_model__min_samples_leaf'],
        random_state=42
    ))
])


In [26]:
# Evaluate using cross_validate

# Evaluate Linear Regression pipeline
lr_tuned_scores = cross_validate(
    estimator=lr_pipeline_tuned,
    X=X,
    y=y,
    cv=5,
    scoring=['neg_mean_squared_error', 'r2']
)

# get the MSE
lr_tuned_mse = f"{-lr_tuned_scores['test_neg_mean_squared_error'].mean():.3f}"

# get the R²
lr_tuned_r2 = f"{lr_tuned_scores['test_r2'].mean():.3f}"

# Evaluate Random Forest Regressor pipeline
rf_tuned_scores = cross_validate(
    estimator=rf_pipeline_tuned,
    X=X,
    y=y,
    cv=5,
    scoring=['neg_mean_squared_error', 'r2']
)

# get the MSE
rf_tuned_mse = f"{-rf_tuned_scores['test_neg_mean_squared_error'].mean():.3f}"

# get the R²
rf_tuned_r2 = f"{rf_tuned_scores['test_r2'].mean():.3f}"

# display the results in a table
results = [
    ["Linear Regression", lr_tuned_mse, lr_tuned_r2],
    ["Random Forest", rf_tuned_mse, rf_tuned_r2]
]

# Print the table
print(tabulate(results, headers=["Model Name (Tuned)", "MSE", "R²"], tablefmt="pretty"))

+--------------------+---------+-------+
| Model Name (Tuned) |   MSE   |  R²   |
+--------------------+---------+-------+
| Linear Regression  | 819.000 | 0.393 |
|   Random Forest    | 404.396 | 0.716 |
+--------------------+---------+-------+


In [27]:
# Evaluate using Train test split
# Using train_test_split

# Linear Regression
lr_pipeline_tuned.fit(X_train, y_train)

# predict using the test set
y_pred_lr_tuned = lr_pipeline_tuned.predict(X_test)

# Evaluate
lr_tuned_mse = mean_squared_error(y_test, y_pred_lr_tuned)
lr_tuned_r2 = r2_score(y_test, y_pred_lr_tuned)

# Random Forest
rf_pipeline_tuned.fit(X_train, y_train)

# predict using the test set
y_pred_rf_tuned = rf_pipeline_tuned.predict(X_test)

# Evaluate
rf_mse_tuned = mean_squared_error(y_test, y_pred_rf_tuned)
rf_r2_tuned = r2_score(y_test, y_pred_rf_tuned)

# display the results in a table
results = [
    ["Linear Regression", lr_tuned_mse, lr_tuned_r2],
    ["Random Forest", rf_mse_tuned, rf_r2_tuned]
]

# Print the table
print(tabulate(results, headers=["Model Name (Tuned)", "MSE", "R²"], tablefmt="pretty"))


+--------------------+-------------------+---------------------+
| Model Name (Tuned) |        MSE        |         R²          |
+--------------------+-------------------+---------------------+
| Linear Regression  | 913.8776823152906 | 0.41194670397097144 |
|   Random Forest    | 418.2113526345291 | 0.7308933469844312  |
+--------------------+-------------------+---------------------+


**Observations:**
- From both tuned results, Random Forest model still outperforms the Linear Regression model.
- The Linear Regression model showed a slight decrease in the R2 score and an increase in the MSE value.
- The Random Forest model showed a slight increase in the R2 score and a decrease in the MSE value.

In [28]:
# export the best performing model

joblib.dump(rf_pipeline, 'salary_prediction_pipeline.pkl')

['salary_prediction_pipeline.pkl']